1. **Transfer learning**
> - finetuning
> - feature extraction

<img src='https://pennylane.ai/qml/_images/transfer_learning_general.png' width="500">

*https://pennylane.ai/qml/demos/tutorial_quantum_transfer_learning.html*

Transfer learning helps in utilising the information learnt by the network in one task/ application 
for a newer task/application
> - helps in faster convergence
    >- enables efficient learning with smaller datasets 

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
import gdown

In [3]:
# Data
!gdown --fuzzy https://drive.google.com/file/d/1gj0DqYGPbj8eU9Wfl_KEDJ79UTzaSp5M/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1gj0DqYGPbj8eU9Wfl_KEDJ79UTzaSp5M
To: /content/rps_stop_phonemes_bdg.gz
100% 41.4M/41.4M [00:00<00:00, 55.4MB/s]


In [4]:
!tar -xf rps_stop_phonemes_bdg.gz

In [5]:
!mv data/rjw0/powerspace/pytorch_CNN_RPS/data/clag_stop_bdg/  clag_stop_bdg

In [6]:
## resize with data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    'clag_stop_bdg/train',
    target_size=(256, 256),
    batch_size=32
)
validation_generator = val_datagen.flow_from_directory(
    'clag_stop_bdg/dev',
    target_size=(256, 256),
    batch_size=32
)
test_generator = val_datagen.flow_from_directory(
    'clag_stop_bdg/test',
    target_size=(256, 256),
    batch_size=32
)

Found 5816 images belonging to 3 classes.
Found 615 images belonging to 3 classes.
Found 312 images belonging to 3 classes.


In [16]:
num_classes=train_generator.num_classes
nb_train_samples = train_generator.samples
nb_val_samples = validation_generator.samples

In [9]:
base_model=tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256, 256, 3))

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
base_model.layers[-1]

In [ ]:
base_model.layers[3].output

<KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'block1_pool')>

In [ ]:
base_model.layers[11].output

<KerasTensor: shape=(None, 32, 32, 256) dtype=float32 (created by layer 'block3_pool')>

In [ ]:
base_model.output

<KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'block5_pool')>

In [ ]:
base_model.summary()

In [10]:
def prepare_VGG_model_for_finetuning(freeze_baselayers=True):
    """
    input_shape: When pretrained networks are utilised, the input shape for the network is generally 
    dependent on the shape of the data the pretrained networks were originally trained. It allows us the exact use of weights.++
    """
    base_model=tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256, 256, 3))    # freeze layers
    if freeze_baselayers==True:
        for layer in base_model.layers:
            layer.trainable=False
    # change here - take the output from different layers of the base model. Eg. base_model.output, base_model.layers[11].output
    x =  base_model.output
    # x = tf.keras.layers.GlobalAveragePooling2D(2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dense(num_classes)(x)
    prediction_layer = tf.keras.layers.Softmax()(x)

    model_new = tf.keras.Model(inputs=base_model.input, outputs=prediction_layer)
    return model_new

In [11]:
model = prepare_VGG_model_for_finetuning(freeze_baselayers=True)

Our problem is classifying the stop sounds that has limited examples for each class. We'd perform transfer learning from a better generic neural network model that is trained with much larger amounts of training data at various levels/layers of the model.

| | side A (finetuning) | side B ( feature extraction ) |
|---|---|---|
|layer 1 |  |  | 
| layer2 | | |
|layer 3 | |  |

The learning rate and optimizers can be one of ( 0.01, 0.0001, 0.0003m, etc. ) and 'Adagrad', 'Adam', 'Adamax', 'Nadam', 'Optimizer', 'RMSprop', 'SGD')

In [14]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [17]:
history = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//32, 
                   epochs=1, validation_data=validation_generator,
                    validation_steps=nb_val_samples/32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


181/181 [==============================] - 108s 516ms/step - loss: 1.1754 - accuracy: 0.3750 - val_loss: 1.1466 - val_accuracy: 0.4049


Ref: 
- https://play.google.com/books/reader?id=mGSGDwAAQBAJ&hl=en&pg=GBS.PA43
- https://keras.io/guides/transfer_learning/